### <center>MLZC Mid-Term project - Q4/2022<img align="right" width="60" height="60" src="../media/heart.png">

# mid 16 Feature importance

<div class="alert alert-block alert-warning">
<b>Warning: </b>Using the <b>Train Full</b> dataset to compare the Features (number of rows is small: 299)
<i>and it is what has been used in our lesson ;-)</i>
</div>

In [1]:
import pandas as pda
import numpy as npy

npy.__version__ , pda.__version__

('1.23.4', '1.5.1')

In [2]:
import pickle
from IPython.display import display

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.__version__ 

'0.12.1'

In [4]:
from sklearn.metrics import mutual_info_score
from sklearn.metrics import roc_auc_score

#### Reload data from previous notebooks

In [5]:
# use TRAin FULL dataset
strFil = '.\pickle\dstTraFul.bin'
with open(strFil, 'rb') as filIn: 
    dfrTraFul, y_TraFul = pickle.load(filIn)
dfrTraFul.shape , len(y_TraFul)

((239, 13), 239)

In [6]:
strFil = '.\pickle\strTar.bin'
with open(strFil, 'rb') as filIn: 
    strTar = pickle.load(filIn)
strTar

'death_event'

In [7]:
strFil = '.\pickle\lstFeaCat.bin'
with open(strFil, 'rb') as filIn: 
    lstFeaCat = pickle.load(filIn)
lstFeaCat

['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

In [8]:
strFil = '.\pickle\lstFeaNum.bin'
with open(strFil, 'rb') as filIn: 
    lstFeaNum = pickle.load(filIn)
lstFeaNum

['age',
 'creatinine_phosphokinase',
 'ejection_fraction',
 'platelets',
 'serum_creatinine',
 'serum_sodium',
 'time']

## Risks of death by Categorical Features 

In [9]:
dead_risk = y_TraFul.mean()
dead_risk.round(5)

0.3431

In [10]:
# double check ;-)
dead_risk = dfrTraFul[strTar].mean()
dead_risk.round(5)

0.3431

In [11]:
# store the difference of Risk in a dataFrame to sort later
dfrRsk = pda.DataFrame(columns=['RskDif'])

# evaluate each CAT column regarding the dead risk
for col in lstFeaCat:
    #print(col)
    df_group = dfrTraFul.groupby(col)[strTar].agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - dead_risk
    df_group['risk'] = df_group['mean'] / dead_risk
    display(df_group)
    floRskDif = df_group['risk'][1] - df_group['risk'][0]
    print(col , 'difference in dead_risk : ' , floRskDif.round(3) )
    dfrRsk.loc[col] = [floRskDif]
    print('_'*50)

,mean,count,diff,risk
anaemia,,,,
0,0.325758,132,-0.017339,0.949464
1,0.364486,107,0.021390,1.062343


anaemia difference in dead_risk :  0.113
__________________________________________________


,mean,count,diff,risk
diabetes,,,,
0,0.330935,139,-0.012161,0.964555
1,0.360000,100,0.016904,1.049268


diabetes difference in dead_risk :  0.085
__________________________________________________


,mean,count,diff,risk
high_blood_pressure,,,,
0,0.324841,157,-0.018255,0.946792
1,0.378049,82,0.034953,1.101874


high_blood_pressure difference in dead_risk :  0.155
__________________________________________________


,mean,count,diff,risk
sex,,,,
0,0.337349,83,-0.005747,0.983250
1,0.346154,156,0.003058,1.008912


sex difference in dead_risk :  0.026
__________________________________________________


,mean,count,diff,risk
smoking,,,,
0,0.347305,167,0.004209,1.012268
1,0.333333,72,-0.009763,0.971545


smoking difference in dead_risk :  -0.041
__________________________________________________


In [12]:
dfrRsk.sort_values(by='RskDif' , ascending=False)

,RskDif
high_blood_pressure,0.155082
anaemia,0.112879
diabetes,0.084713
sex,0.025662
smoking,-0.040723


<div class="alert alert-block alert-warning">
In this dataset, for our categorical features :
<ul>
<li><b>high_blood_pressure</b> increases the risk of death by the most importance.
<li><b>anemia and diabetes</b> increase also the risk of death.  
<li><b>sex and smoking</b> do not influence much the risk of death (amazing for smoking!).   
</ul>
</div>

# Categorical Features importance (Mutual information)

In [13]:
mutual_info_score(dfrTraFul[strTar], dfrTraFul['high_blood_pressure'] )

0.0014059051620281537

In [14]:
# store the difference of Risk in a dataFrame to sort later
dfrMui = pda.DataFrame(columns=['MutInf'])

# evaluate each CAT column regarding the dead risk
for col in lstFeaCat:
    #print(col)
    floMutInf = mutual_info_score(dfrTraFul[strTar], dfrTraFul[col] )
    dfrMui.loc[col] = [floMutInf]
    print(col , 'Mutual information vs dead_risk : ' , floMutInf.round(6) )
    
dfrMui.sort_values(by='MutInf' , ascending=False)

anaemia Mutual information vs dead_risk :  0.000822
diabetes Mutual information vs dead_risk :  0.000455
high_blood_pressure Mutual information vs dead_risk :  0.001406
sex Mutual information vs dead_risk :  3.9e-05
smoking Mutual information vs dead_risk :  9.1e-05


,MutInf
high_blood_pressure,0.001406
anaemia,0.000822
diabetes,0.000455
smoking,0.000091
sex,0.000039


# Correlation of numerical features

#### corrwith

In [15]:
dfrTraFul[lstFeaNum].corrwith(dfrTraFul[strTar]).sort_values(ascending=False)

serum_creatinine            0.312228
age                         0.226820
creatinine_phosphokinase    0.085204
platelets                  -0.069756
serum_sodium               -0.203004
ejection_fraction          -0.252100
time                       -0.519556
dtype: float64

#### abs(corrwith)

In [17]:
dfrTraFul[lstFeaNum].corrwith(dfrTraFul[strTar]).abs().sort_values(ascending=False)

time                        0.519556
serum_creatinine            0.312228
ejection_fraction           0.252100
age                         0.226820
serum_sodium                0.203004
creatinine_phosphokinase    0.085204
platelets                   0.069756
dtype: float64

# Correlation of log1p(numerical features)

#### corrwith( log1p( ) )

In [18]:
npy.log1p(dfrTraFul[lstFeaNum]).corrwith(dfrTraFul[strTar]).sort_values(ascending=False)

serum_creatinine            0.359451
age                         0.215981
creatinine_phosphokinase    0.057817
platelets                  -0.087695
serum_sodium               -0.202032
ejection_fraction          -0.299468
time                       -0.567556
dtype: float64

#### abs( corrwith( log1p( ) ) )

In [19]:
npy.log1p(dfrTraFul[lstFeaNum]).corrwith(dfrTraFul[strTar]).abs().sort_values(ascending=False)

time                        0.567556
serum_creatinine            0.359451
ejection_fraction           0.299468
age                         0.215981
serum_sodium                0.202032
platelets                   0.087695
creatinine_phosphokinase    0.057817
dtype: float64

<div class="alert alert-block alert-warning">
<b>log1p :  </b>3 features seems to give better correlation with log1p:
<ul>
<li>time :                      0.567556
<li>serum_creatinine :          0.359451
<li>ejection_fraction :         0.299468
<li>platelets :                 0.087695
</div>

# Correlation of Categorical Features
### Confirm previous analysys ;-)

In [20]:
# try it with Categorical too (confirms previous findings -in order of importance-)
dfrTraFul[lstFeaCat].corrwith(dfrTraFul[strTar]).abs().sort_values(ascending=False)

high_blood_pressure    0.053208
anaemia                0.040565
diabetes               0.030201
smoking                0.013503
sex                    0.008830
dtype: float64

# Correlation of *all* Features

#### corrwith()

In [21]:
serCorFea = dfrTraFul[ lstFeaCat + lstFeaNum ].corrwith(dfrTraFul[strTar]).sort_values(ascending=False)
serCorFea

serum_creatinine            0.312228
age                         0.226820
creatinine_phosphokinase    0.085204
high_blood_pressure         0.053208
anaemia                     0.040565
diabetes                    0.030201
sex                         0.008830
smoking                    -0.013503
platelets                  -0.069756
serum_sodium               -0.203004
ejection_fraction          -0.252100
time                       -0.519556
dtype: float64

#### abs( corrwith() )

In [22]:
serCorFea = dfrTraFul[ lstFeaCat + lstFeaNum ].corrwith(dfrTraFul[strTar]).abs().sort_values(ascending=False)
serCorFea

time                        0.519556
serum_creatinine            0.312228
ejection_fraction           0.252100
age                         0.226820
serum_sodium                0.203004
creatinine_phosphokinase    0.085204
platelets                   0.069756
high_blood_pressure         0.053208
anaemia                     0.040565
diabetes                    0.030201
smoking                     0.013503
sex                         0.008830
dtype: float64

In [23]:
# store the Features in a dataFrame to use later
dfrFea = pda.DataFrame(columns=['FeaTyp' , 'Cor' , 'CorAbs' , 'Log' , 'LogBst' , 'LogCor' , 'LogCorAbs'])


# for each feature calculate Correlation.
# for each numercic feature calculate also the Correlation on the log1p(feature)
# mark it "+" if log1p(fea) is better and indicate the percentage of improvement
for fea in lstFeaCat + lstFeaNum:
    
    floCor = dfrTraFul[[ fea] ].corrwith(dfrTraFul[strTar])[0]
    
    if fea in lstFeaCat:
        feaTyp = 'Cat'
        floCorLog = npy.nan
        LogBst = npy.nan
        floBst = npy.nan
    else:
        feaTyp = 'Num'
        floCorLog = npy.log1p(dfrTraFul[ [fea] ] ).corrwith(dfrTraFul[strTar])[0]
        # log1p better?
        if abs(floCor) < abs(floCorLog):
            LogBst = '+'
            floBst = ( abs(floCorLog) - abs(floCor) ) / abs(floCor) 
        else:
            LogBst = '-'
            floBst = ( abs(floCorLog) - abs(floCor) ) / abs(floCor) 
    
    print(fea , '(', feaTyp , ') :' , floCor , ' - ' , abs(floCor) , 
           '\n         log1p:' ,  LogBst , round(floBst,2)*100 , floCorLog , abs(floCorLog) )
    
    dfrFea.loc[fea] = [ feaTyp, floCor , abs(floCor) , LogBst , round(floBst,2)*100 , floCorLog , abs(floCorLog) ]

dfrFea.sort_values(by='CorAbs' , ascending=False)   

anaemia ( Cat ) : 0.04056502234023351  -  0.04056502234023351 
         log1p: nan nan nan nan
diabetes ( Cat ) : 0.030200714138686517  -  0.030200714138686517 
         log1p: nan nan nan nan
high_blood_pressure ( Cat ) : 0.053208016156594595  -  0.053208016156594595 
         log1p: nan nan nan nan
sex ( Cat ) : 0.008829716375519618  -  0.008829716375519618 
         log1p: nan nan nan nan
smoking ( Cat ) : -0.013502930910508273  -  0.013502930910508273 
         log1p: nan nan nan nan
age ( Num ) : 0.22681984868703398  -  0.22681984868703398 
         log1p: - -5.0 0.2159810909422658 0.2159810909422658
creatinine_phosphokinase ( Num ) : 0.08520413190517025  -  0.08520413190517025 
         log1p: - -32.0 0.05781666705109902 0.05781666705109902
ejection_fraction ( Num ) : -0.2520997642755393  -  0.2520997642755393 
         log1p: + 19.0 -0.29946841764049315 0.29946841764049315
platelets ( Num ) : -0.06975556099096998  -  0.06975556099096998 
         log1p: + 26.0 -0.087694692512028

,FeaTyp,Cor,CorAbs,Log,LogBst,LogCor,LogCorAbs
time,Num,-0.519556,0.519556,+,9.0,-0.567556,0.567556
serum_creatinine,Num,0.312228,0.312228,+,15.0,0.359451,0.359451
ejection_fraction,Num,-0.252100,0.252100,+,19.0,-0.299468,0.299468
age,Num,0.226820,0.226820,-,-5.0,0.215981,0.215981
serum_sodium,Num,-0.203004,0.203004,-,-0.0,-0.202032,0.202032
creatinine_phosphokinase,Num,0.085204,0.085204,-,-32.0,0.057817,0.057817
platelets,Num,-0.069756,0.069756,+,26.0,-0.087695,0.087695
high_blood_pressure,Cat,0.053208,0.053208,NaN,NaN,NaN,NaN
anaemia,Cat,0.040565,0.040565,NaN,NaN,NaN,NaN
diabetes,Cat,0.030201,0.030201,NaN,NaN,NaN,NaN


In [24]:
# List of all features
dfrFea.index.values.tolist()

['anaemia',
 'diabetes',
 'high_blood_pressure',
 'sex',
 'smoking',
 'age',
 'creatinine_phosphokinase',
 'ejection_fraction',
 'platelets',
 'serum_creatinine',
 'serum_sodium',
 'time']

In [25]:
# List of all features sorted by Abs of correlation to death
dfrFea.sort_values(by='CorAbs' , ascending=False).index.values.tolist()

['time',
 'serum_creatinine',
 'ejection_fraction',
 'age',
 'serum_sodium',
 'creatinine_phosphokinase',
 'platelets',
 'high_blood_pressure',
 'anaemia',
 'diabetes',
 'smoking',
 'sex']

In [26]:
# List of all Categorical features sorted by Abs of correlation to death
dfrFea[dfrFea.FeaTyp == 'Cat'].sort_values(by='CorAbs' , ascending=False).index.values.tolist()

['high_blood_pressure', 'anaemia', 'diabetes', 'smoking', 'sex']

In [27]:
# List of all Numerical features sorted by Abs of correlation to death
dfrFea[dfrFea.FeaTyp == 'Num'].sort_values(by='CorAbs' , ascending=False).index.values.tolist()

['time',
 'serum_creatinine',
 'ejection_fraction',
 'age',
 'serum_sodium',
 'creatinine_phosphokinase',
 'platelets']

# ROC AUC of all Features

In [28]:
# Dataframe to save result index=column name
dfrAuc = pda.DataFrame(columns=[ 'Auc'])

# ROC for each Num Col
#for fea in dfrFea[dfrFea.FeaTyp == 'Num'].index.values.tolist():
for fea in dfrFea.index.values.tolist():

    floAuc = roc_auc_score(y_TraFul, dfrTraFul[fea])
    print( fea ,' floAuc=' , floAuc)
    
    if floAuc < 0.5:
        floAuc = roc_auc_score(y_TraFul, -dfrTraFul[fea])
        print('         ! <0.5 => -floAuc=' , floAuc)
    
    dfrAuc.loc[fea] = [ floAuc ]
    
dfrAuc.sort_values(by='Auc' , ascending=False)

anaemia  floAuc= 0.5212443684946404
diabetes  floAuc= 0.5156905390709958
high_blood_pressure  floAuc= 0.5266040080782973
sex  floAuc= 0.5044275283517167
smoking  floAuc= 0.49347522137641764
         ! <0.5 => -floAuc= 0.5065247786235825
age  floAuc= 0.6160090104085754
creatinine_phosphokinase  floAuc= 0.5262933043343172
ejection_fraction  floAuc= 0.33622028895448186
         ! <0.5 => -floAuc= 0.6637797110455181
platelets  floAuc= 0.4557247164828337
         ! <0.5 => -floAuc= 0.5442752835171664
serum_creatinine  floAuc= 0.7251825384495884
serum_sodium  floAuc= 0.3577753611931024
         ! <0.5 => -floAuc= 0.6422246388068976
time  floAuc= 0.1726736057169489
         ! <0.5 => -floAuc= 0.8273263942830511


,Auc
time,0.827326
serum_creatinine,0.725183
ejection_fraction,0.663780
serum_sodium,0.642225
age,0.616009
platelets,0.544275
high_blood_pressure,0.526604
creatinine_phosphokinase,0.526293
anaemia,0.521244
diabetes,0.515691


In [29]:
dfrFea = dfrFea.join(dfrAuc)

In [30]:
dfrFea.sort_values(by='Auc' , ascending=False)

,FeaTyp,Cor,CorAbs,Log,LogBst,LogCor,LogCorAbs,Auc
time,Num,-0.519556,0.519556,+,9.0,-0.567556,0.567556,0.827326
serum_creatinine,Num,0.312228,0.312228,+,15.0,0.359451,0.359451,0.725183
ejection_fraction,Num,-0.252100,0.252100,+,19.0,-0.299468,0.299468,0.663780
serum_sodium,Num,-0.203004,0.203004,-,-0.0,-0.202032,0.202032,0.642225
age,Num,0.226820,0.226820,-,-5.0,0.215981,0.215981,0.616009
platelets,Num,-0.069756,0.069756,+,26.0,-0.087695,0.087695,0.544275
high_blood_pressure,Cat,0.053208,0.053208,NaN,NaN,NaN,NaN,0.526604
creatinine_phosphokinase,Num,0.085204,0.085204,-,-32.0,0.057817,0.057817,0.526293
anaemia,Cat,0.040565,0.040565,NaN,NaN,NaN,NaN,0.521244
diabetes,Cat,0.030201,0.030201,NaN,NaN,NaN,NaN,0.515691


## Saving object for next notebooks (\\pickle)

In [31]:
strFil = '.\pickle\dfrFea.bin'
with open(strFil, 'wb') as filOut: 
    pickle.dump(dfrFea, filOut)